# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.45MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
%pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [4]:
%pip install -qU langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00


In [5]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00


In [6]:
%pip install -qU duckduckgo-search langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.0 MB/s eta 0:00:00


In [7]:
!pip install -U langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.6 MB/s eta 0:00:00


In [ ]:
import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("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")

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [8]:
import pandas as pd
import numpy as np
from google.colab import userdata
import os
from pydantic import BaseModel, Field

from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [9]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
df.isnull().value_counts()

,,count
المخالفة,الغرامة,
False,False,104


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [11]:
df_list = []
df_list = df.values.tolist()

In [ ]:
df_list = ['المخالفة'] + df_list + ['الغرامة']

In [ ]:
df_list

['المخالفة',
 ['قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.',
  'الغرامة المالية 100 - 150 ريال'],
 ['ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.',
  'الغرامة المالية 100 - 150 ريال'],
 ['عدم وجود تأمين ساري للمركبة.', 'الغرامة المالية 100 - 150 ريال'],
 ['عبور المشاة للطرق من غير الأماكن المخصصة لهم.',
  'الغرامة المالية 100 - 150 ريال'],
 ['عدم تقيد المشاة بالإشارات الخاصة بهم.', 'الغرامة المالية 100 - 150 ريال'],
 ['وقوف المركبة في أماكن غير مخصصة للوقوف.', 'الغرامة المالية 100 - 150 ريال'],
 ['عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.',
  'الغرامة المالية 100 - 150 ريال'],
 ['عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.',
  'الغرامة المالية 150 - 300 ريال'],
 ['الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.',
  'الغرامة المالية 150 - 300 ريال'],
 ['قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.',
  'الغرامة الم

In [12]:
for i in range(len(df_list)):
  df_list[i] = ['المخالفة-الغرامة'] + df_list[i]

In [13]:
df_list

[['المخالفة-الغرامة',
  'قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'عدم وجود تأمين ساري للمركبة.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'عبور المشاة للطرق من غير الأماكن المخصصة لهم.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'عدم تقيد المشاة بالإشارات الخاصة بهم.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'وقوف المركبة في أماكن غير مخصصة للوقوف.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.',
  'الغرامة المالية 100 - 150 ريال'],
 ['المخالفة-الغرامة',
  'عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.',
  'الغرامة المالية 150 - 300 ريال'],
 ['المخالفة-الغرامة',
  'الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين م

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [14]:

documents = [Document(page_content=' '.join(text_list)) for text_list in df_list]
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,
    chunk_overlap=64
)
doc_splits = text_splitter.split_documents(documents)
print(len(doc_splits))

104


## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [15]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.4 MB/s eta 0:00:

In [16]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="akhooli/gpt2-small-arabic")
vectordb = Chroma.from_documents(documents=doc_splits, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-16-0c6327e1884f>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="akhooli/gpt2-small-arabic")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in ju

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [17]:
Template = """
انت مطالب بالاجابة على قيمة الغرامة المتوقعة لكل مخالفه

هنا لديك بعض الأمثلة:
المثال الأول:
سؤال المستخدم:قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.؟
الأجابة:الغرامة المالية من 100-150
'''

المثال الثاني:
سؤال المستخدم:ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.؟
الإجابة:الغرامة المالية من 100-150
'''

المثال الثالث:
سؤال المستخدم:التحرك بالمركبة بسرعة عالية، بحيث تحدث الإطارات صوتاً عالياً.؟
الإجابة: الغرامة المالية من 150-300
'''

المثال الرابع:
سؤال المستخدم:تظليل زجاج المركبة دون التقيد بالضوابط التي تضعها الإدارة المختصة.؟
الإجابة:الغرامة المالية من 500-900

"""





In [35]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=Template, input_variables=["سؤال المستخدم", "الإجابة"]
)



## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [18]:
!pip install langchain_core

In [19]:
!pip install langchain_ollama

In [33]:
!pip install langchain_groq

  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.4 MB/s eta 0:00:00


In [34]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3.1",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,

)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3.1")



## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [29]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="ماهي الغرامة على القيادة بدون رخصة؟"
    ),
]

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [30]:
ai_msg = chat_model.invoke(messages)

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
inpu_user = input("ماهو سؤالك؟")

ماهو سؤالك؟ماهي الغرامة على القيادة بدون رخصة؟


In [ ]:
message.append(HumanMessage(content=inpu_user))

In [ ]:
ai_msg = chat_model.invoke(message)